# Dynamic Treatment Regime

In [1]:
import numpy as np
import scipy.special

# generate dynamic data
def gen_data(alpha, A, B, c, n, d, tau1=None, tau2=None):
    S1 = np.random.normal(0, 1, size=(n, d))
    if tau1 is None:
        D1 = np.random.binomial(1, scipy.special.expit(S1 @ alpha))
    else:
        D1 = tau1 * np.ones(n)
    S2 = S1 @ A.T + D1.reshape(-1, 1) @ B + np.random.normal(0, 1, size=(n, d))
    if tau2 is None:
        D2 = np.random.binomial(1, scipy.special.expit(S2 @ alpha))
    else:
        D2 = tau2 * np.ones(n)
    Y = D2 + S2 @ c + np.random.normal(0, 1, size=(n,))
    return S1, D1, S2, D2, Y

In [2]:
# instance parameters
d = 4
alpha = np.random.uniform(-1, 1, size=d)
A = np.random.uniform(-1, 1, size=(d, d))
B = np.random.uniform(-1, 1, size=(1, d))
c = np.random.uniform(-1, 1, size=d)

In [3]:
# observational treatment regime mean outcome
n = 1000000
np.mean(gen_data(alpha, A, B, c, n, d)[-1])

0.5957640334260048

In [4]:
# expected value of counterfactual treatment regime (0, 0)
n = 1000000
V00 = np.mean(gen_data(alpha, A, B, c, n, d, 0, 0)[-1])
V00

0.0011186618066241865

In [5]:
# expected value of counterfactual treatment regime (1, 1)
n = 1000000
V11 = np.mean(gen_data(alpha, A, B, c, n, d, 1, 1)[-1])
V11

1.2194222032891435

# Generate Observed Data

In [6]:
n = 5000
S1, D1, S2, D2, y = gen_data(alpha, A, B, c, n, d)

# Wrong Identification by Conditioning of Effects with OLS

In [7]:
from sklearn.linear_model import LinearRegression, LogisticRegression

effects = LinearRegression().fit(np.column_stack([D1, D2, S1, S2]), y).coef_[:2]

In [8]:
# estimated expected value of counterfactual treatment regime (0, 0)
V00hat = np.mean(y - effects[0] * D1 - effects[1] * D2)
print(f'Estimate: {V00hat:.4f}, True: {V00:.4f}')

Estimate: 0.1178, True: 0.0011


In [9]:
# estimated expected value of counterfactual treatment regime (1, 1)
V11hat = np.mean(y - effects[0] * (D1 - 1) - effects[1] * (D2 - 1))
print(f'Estimate: {V11hat:.4f}, True: {V11:.4f}')

Estimate: 1.1118, True: 1.2194


# G-Computation

In [10]:
from sklearn.linear_model import LinearRegression, LogisticRegression

def cntf(S1, D1, S2, D2, y, tau1, tau2, modely1, modely2):
    n = S1.shape[0]
    my2 = modely2.fit(np.column_stack([D2, S2]), y)
    Yadj = my2.predict(np.column_stack([tau2 * np.ones(n), S2]))

    my1 = modely1.fit(np.column_stack([D1, S1]), Yadj)
    Yadj = my1.predict(np.column_stack([tau1 * np.ones(n), S1]))

    return np.mean(Yadj)

short_cntf = lambda tau1, tau2: cntf(S1, D1, S2, D2, y, tau1, tau2, LinearRegression(), LinearRegression())

In [11]:
print(f'Estimate: {short_cntf(0, 0):.4f}, True: {V00:.4f}')

Estimate: 0.0081, True: 0.0011


In [12]:
print(f'Estimate: {short_cntf(1, 1):.4f}, True: {V11:.4f}')

Estimate: 1.2160, True: 1.2194


# Doubly (Multiply) Robust Estimation and Confidence Interval

In [13]:
from sklearn.linear_model import LinearRegression, LogisticRegression

def drcntf(S1, D1, S2, D2, y, tau1, tau2, modely1, modely2, modeld1, modeld2):
    n = S1.shape[0]
    
    my2 = modely2.fit(np.column_stack([D2, S2]), y)
    resy2 = y - my2.predict(np.column_stack([D2, S2]))
    Yadj2 = my2.predict(np.column_stack([tau2 * np.ones(n), S2]))
    
    md2 = modeld2.fit(S2, D2)
    prop2 = md2.predict_proba(S2)[:, 1]
    ips2 = tau2 * (D2 == 1) / prop2 + (1 - tau2) * (D2 == 0) / (1 - prop2)

    my1 = modely1.fit(np.column_stack([D1, S1]), Yadj2)
    resy1 = Yadj2 - my1.predict(np.column_stack([D1, S1]))
    Yadj1 = my1.predict(np.column_stack([tau1 * np.ones(n), S1]))

    md1 = modeld1.fit(S1, D1)
    prop1 = md1.predict_proba(S1)[:, 1]
    ips1 = tau1 * (D1 == 1) / prop1 + (1 - tau1) * (D1 == 0) / (1 - prop1)

    dr = Yadj1 + ips1 * resy1 + ips1 * ips2 * resy2
    return np.mean(dr), np.std(dr) / np.sqrt(n)

short_drcntf = lambda tau1, tau2: drcntf(S1, D1, S2, D2, y, tau1, tau2,
                                         LinearRegression(), LinearRegression(),
                                         LogisticRegression(C=1000), LogisticRegression(C=1000))

In [14]:
V00hat, V00se = short_drcntf(0, 0)
print(f'Estimate: {V00hat:.4f} ({V00se:.4f}), True: {V00:.4f}')

Estimate: -0.0311 (0.0488), True: 0.0011


In [15]:
V11hat, V11se = short_drcntf(1, 1)
print(f'Estimate: {V11hat:.4f} ({V11se:.4f}), True: {V11:.4f}')

Estimate: 1.2167 (0.0542), True: 1.2194


# G-Estimation: Dynamic Partialling Out (Neyman Orthogonal)

In [16]:
from sklearn.linear_model import LinearRegression, LogisticRegression
import scipy.linalg

def dynplr(S1, D1, S2, D2, y, tau1, tau2, modely1, modely2, modeld1, modeld2):
    n = S1.shape[0]

    psi = np.zeros((n, 3))
    J = np.zeros((3, 3))
    
    # Estimate second period treatment effect using Partialling Out
    my2 = modely2.fit(S2, y)
    resy2 = y - my2.predict(S2)
    md2 = modeld2.fit(S2, D2)
    resD2 = D2 - md2.predict_proba(S2)[:, 1]
    delta2 = np.mean(resy2 * resD2) / np.mean(resD2**2) # effect of D2
    
    # moment function for delta2
    psi[:, 2] = (resy2 - delta2 * resD2) * resD2
    # jacobian of moment
    J[2, 2] = - np.mean(resD2**2)

    # Adjust the outcome for second period treatment
    Yadj2 = y - delta2 * (D2 - tau2)

    # Estimate first period treatment effect using Partialling Out
    my1 = modely1.fit(S1, Yadj2)
    resy1 = Yadj2 - my1.predict(S1)
    md1 = modeld1.fit(S1, D1)
    resD1 = D1 - md1.predict_proba(S1)[:, 1]
    delta1 = np.mean(resy1 * resD1) / np.mean(resD1**2) # effect of D1
    
    # moment function for delta1
    psi[:, 1] = (resy1 - delta1 * resD1) * resD1
    # jacobian of moment
    J[1, 1:] = - np.array([np.mean(resD1**2), np.mean(resD1 * (D1 - tau1))])
    
    # Adjust the outcome for first period treatment
    Yadj1 = Yadj2 - delta1 * (D1 - tau1)
    
    # Mean counterfactual value estimate
    theta = np.mean(Yadj1)
    
    # moment for theta
    psi[:, 0] = Yadj1 - theta
    # jacobian of moment
    J[0, :] = - np.array([1, np.mean(D1 - tau1), np.mean(D2 - tau2)])
    
    # calculating covariance of (theta, delta1, delta2)
    Jinv = scipy.linalg.pinv(J)
    Sigma = psi.T @ psi / n
    V = Jinv @ Sigma @ Jinv.T

    return theta, np.sqrt(V[0, 0] / n)

short_dynplr = lambda tau1, tau2: dynplr(S1, D1, S2, D2, y, tau1, tau2,
                                         LinearRegression(), LinearRegression(),
                                         LogisticRegression(C=1000), LogisticRegression(C=1000))

In [17]:
V00hat, V00se = short_dynplr(0, 0)
print(f'Estimate: {V00hat:.4f} ({V00se:.4f}), True: {V00:.4f}')

Estimate: 0.0149 (0.0407), True: 0.0011


In [18]:
V11hat, V11se = short_dynplr(1, 1)
print(f'Estimate: {V11hat:.4f} ({V11se:.4f}), True: {V11:.4f}')

Estimate: 1.2094 (0.0404), True: 1.2194


# Project Star Example Application

Project star was a longitudinal study where students were randomized in small or regular sized classrooms from kindergarten till grade 4. We will use here the first two classes, k and grade 1 and examine the effect of the small classroom interventions in kindergarden and grade 1 on test scores at the end of grade 1.

In [19]:
import pandas as pd 
URL = 'https://raw.githubusercontent.com/gsbDBI/ExperimentData/master/Project%20STAR/STAR_Students.tab'
df = pd.read_csv(URL, delimiter='\t')

In [20]:
S1cols = ['gender', 'race', 'birthmonth', 'birthyear',
        'gkschid', 'gksurban', 'gktgen',
        'gktrace', 'gkthighdegree', 'gktcareer',
        'gktyears', 'gkfreelunch', 'gkrepeat',
        'gkspeced', 'gkspecin']
D1cols = ['gkclasstype']
y1cols = ['gktreadss', 'gktmathss']
S2cols = ['gender', 'race', 'birthmonth', 'birthyear',
        'g1schid', 'g1surban', 'g1tgen',
        'g1trace', 'g1thighdegree', 'g1tcareer',
        'g1tyears', 'g1freelunch',
        'g1speced', 'g1specin']
D2cols = ['g1classtype']
y2cols = ['g1treadss', 'g1tmathss']
df = df[S1cols + D1cols + y1cols + S2cols + D2cols + y2cols]
df = df.dropna()

In [21]:
D1 = 1.0 * (df[D1cols].values.flatten() == 1) # is small class
S1 = df[S1cols]
D2 = 1.0 * (df[D2cols].values.flatten() == 1) # is small class
S2 = df[S2cols + S1cols + y1cols + D1cols]
y = np.sum(df[y2cols].values, axis=1) # total of reading and math scores

In [22]:
from sklearn.linear_model import LassoCV, LogisticRegressionCV
import warnings
warnings.simplefilter('ignore')
short_drcntf = lambda tau1, tau2: drcntf(S1, D1, S2, D2, y, tau1, tau2,
                                         LassoCV(), LassoCV(),
                                         LogisticRegressionCV(), LogisticRegressionCV())

In [23]:
for tau1 in [0, 1]:
    for tau2 in [0, 1]:
        Vhat, Vse = short_drcntf(tau1, tau2)
        print(f'Estimate({tau1}, {tau2}): {Vhat:.4f} ({Vse:.4f})')

Estimate(0, 0): 1058.1835 (2.4596)
Estimate(0, 1): 1063.9774 (1.5556)
Estimate(1, 0): 1072.5690 (1.7907)
Estimate(1, 1): 1090.5339 (6.4301)


In [24]:
short_dynplr = lambda tau1, tau2: dynplr(S1, D1, S2, D2, y, tau1, tau2,
                                         LassoCV(), LassoCV(),
                                         LogisticRegressionCV(), LogisticRegressionCV())

In [25]:
for tau1 in [0, 1]:
    for tau2 in [0, 1]:
        Vhat, Vse = short_dynplr(tau1, tau2)
        print(f'Estimate({tau1}, {tau2}): {Vhat:.4f} ({Vse:.4f})')

Estimate(0, 0): 1059.2718 (1.8834)
Estimate(0, 1): 1069.4421 (2.4312)
Estimate(1, 0): 1070.8131 (2.4958)
Estimate(1, 1): 1080.9834 (2.7793)
